In [2]:
import numpy
import pandas as pd

In [8]:
r = pd.read_csv('~/Desktop/small.csv', engine='python', error_bad_lines = False)

r

Skipping line 2: ',' expected after '"'
Skipping line 3: ',' expected after '"'
Skipping line 4: unexpected end of data


,id,text,lead_id


In [22]:
markup = '<a href="http://example.com/">\nI li<asdfadsb>nked to <i>example.com</i>\n</a>'

soup = BeautifulSoup(markup)

soup.i.get_text()

/Users/ewanog/Documents/work/code/venv/p3/lib/python3.6/site-packages/bs4/__init__.py:181: UserWarning: No parser was explicitly specified, so I'm using the best available HTML parser for this system ("html5lib"). This usually isn't a problem, but if you run this code on another system, or in a different virtual environment, it may use a different parser and behave differently.

The code that caused this warning is on line 193 of the file /usr/local/Cellar/python3/3.6.1/Frameworks/Python.framework/Versions/3.6/lib/python3.6/runpy.py. To get rid of this warning, change code that looks like this:

 BeautifulSoup([your markup])

to this:

 BeautifulSoup([your markup], "html5lib")

  markup_type=markup_type))


'example.com'

In [15]:
from bs4 import BeautifulSoup

html = "<text>&lt;h1&gt;Central African Republic cannot face another year abandoned&lt;/h1&gt;&lt;html&gt;&lt;body&gt;&lt;div&gt;&lt;div class=&quot;field body&quot;&gt;&lt    ;p&gt;Ahead of an international donor meeting in Brussels on Thursday, Oxfam warns that the people of Central African Republic cannot face another year of the present dire     situation which has left half of the 4.8 million population in need of humanitarian assistance. The UN appeal fund is still only 32% funded as we near the end of this year.     One of the poorest countries in the world, the people are in extreme need: two million people are struggling to find enough to eat; 65% of the population lacks access to s    afe drinking water; more than 800,000 people have been forced from their homes, either internally displaced within the country or driven into neighboring countries - all of     them having to start new lives from nothing.&lt;/p&gt;&lt;p&gt;Women have been particularly affected by the conflict: last year an estimated 60,000 women experienced viole    nce, meaning that one woman was raped or abused every 15 minutes.&lt;/p&gt;&lt;p&gt;Ferran Puig, Oxfam's Central African Republic Country Director, said: <93>Despite peacef    ul presidential and legislative elections earlier this year, a large part of the territory remains under the control of armed groups which continue to perpetrate violence a    gainst civilians. Recently, a new wave of violence in the capital, Bangui, and in the north of the country has rapidly aggravated the security situation - 37 people were ki    lled in October in an attack on a camp for displaced people in Kaga Bandoro, 300 Km from Bangui.&lt;/p&gt;&lt;p&gt;&quot;Over the last 5 years, four donor meetings held in     Brussels have simply resulted in unmet pledges leaving the country to relapse into new crises. As well as a bigger humanitarian response, we need to see peace building and     development work that can take a longer term approach for the rebuilding of the country. Donors need to provide the necessary funds immediately.<94>&lt;/p&gt;&lt;p&gt;Oxfam     says Thursday's donor meeting has the potential to be an important step in restoring peace and security, and reviving the economy, but that donors need to show both a long     term commitment and provide flexible funding. The organization has been working in CAR since 2014 with projects on food security, livelihoods, protection and gender, water    , sanitation and hygiene in the capital, Bangui, and in Bria, Paoua and Batangafo, reaching more than 300,000 people.&lt;/p&gt;&lt;/div&gt;&lt;/div&gt;&lt;/body&gt;&lt;/htm    l&gt;</text>"

soup = BeautifulSoup(html)

soup.get_text()

/Users/ewanog/Documents/work/code/venv/p3/lib/python3.6/site-packages/bs4/__init__.py:181: UserWarning: No parser was explicitly specified, so I'm using the best available HTML parser for this system ("html5lib"). This usually isn't a problem, but if you run this code on another system, or in a different virtual environment, it may use a different parser and behave differently.

The code that caused this warning is on line 193 of the file /usr/local/Cellar/python3/3.6.1/Frameworks/Python.framework/Versions/3.6/lib/python3.6/runpy.py. To get rid of this warning, change code that looks like this:

 BeautifulSoup([your markup])

to this:

 BeautifulSoup([your markup], "html5lib")

  markup_type=markup_type))


'<h1>Central African Republic cannot face another year abandoned</h1><html><body><div><div class="field body"><    ;p>Ahead of an international donor meeting in Brussels on Thursday, Oxfam warns that the people of Central African Republic cannot face another year of the present dire     situation which has left half of the 4.8 million population in need of humanitarian assistance. The UN appeal fund is still only 32% funded as we near the end of this year.     One of the poorest countries in the world, the people are in extreme need: two million people are struggling to find enough to eat; 65% of the population lacks access to s    afe drinking water; more than 800,000 people have been forced from their homes, either internally displaced within the country or driven into neighboring countries - all of     them having to start new lives from nothing.</p><p>Women have been particularly affected by the conflict: last year an estimated 60,000 women experienced viole    nce, meaning that on

In [4]:
"""for pulling down data from hr.info API on response level and sending to CSV"""
#locations: https://api.hpc.tools/v1/public/location
#clusters: https://api.hpc.tools/v1/public/global-cluster ["code":"SHL"]
#https://api.hpc.tools/docs/v1/
#boundary cat by looping through locations

#TODO: subtract outgoing

import urllib.request, json
import re
from itertools import groupby

import pandas as pd
import numpy as np
from pandas.io.json import json_normalize

import utils


class fts(object):
    def __init__(self, sc, test=None, all_sector=False):
        self.all_sector = all_sector
        self.test = test
        self.sc = sc

        self.fts = self.master_pull()
        # self.merge_sc()
        self.flows = self.extract_flows()

        # remove flows col as it's quite large
#         self.fts = self.fts.drop('fts.data.flows', axis=1)

    def master_pull(self):
        """group by year, country and pull down total values with optional filter by cluster.
            returns: dict of {sector 1 : info df}
        """
        SHELTER_ABBRV = 'SHL'

        if self.test:
            self.hist = self.sc[:self.test]
        else:
            self.hist = self.sc

        #create cluster_vals which will be a dict of {cluster : df}
        """
            AGR : Agriculture
            CCM : Camp Coordination / Management
            PRO-CPN : Child Protection
            CSS : Coordination and support services
            ERY : Early Recovery
            EDU : Education
            SHL : Emergency Shelter and NFI
            TEL : Emergency Telecommunications
            FSC : Food Security
            PRO-GBV : Gender Based Violence
            HEA : Health
            PRO-HLP : Housing, Land and Property
            LOG : Logistics
            PRO-MIN : Mine Action
            MS : Multi-sector
            NUT : Nutrition
            OTH : Other
            PRO : Protection
            WSH : Water Sanitation Hygiene
        """
        clusters = [SHELTER_ABBRV]
        if self.all_sector:
            clusters = [v['code'] for v in json_normalize(json.loads \
                                                              (urllib.request.urlopen(
                                                                "https://api.hpc.tools/v1/public/global-cluster")
                                                                .read().decode()))['data'][0]]

        #for each cluster, prep string for each URL and get year
        URL_STR = 'https://api.hpc.tools/v1/public/fts/flow?countryISO3={0}&year={1} &globalClusterCode={2}'
        hrefz = []
        for clust in clusters:
            self.hist.apply(lambda v: hrefz.append(URL_STR.format(v['sc.iso3'], v['sc.year'], clust)) , axis = 1)

        # now submit the URLs to the API service and concat together and
        # then assign to cluster_values using cluser code in URL
        pulled = None
        for v in utils.api_pull(hrefz, False):
            ins_df = json_normalize(v['data'])

            ins_df['sector'] = re.match('.*globalClusterCode=(.*)', v['url']).groups()[0]
            ins_df['year'] = re.match('.*year=([0-9]{4})', v['url']).groups()[0]
            ins_df['iso3'] = re.match('.*countryISO3=(.{3})', v['url']).groups()[0]

            if pulled is None:
                pulled = ins_df

            else:
                pulled = pd.concat([pulled, ins_df])

        return pulled.add_prefix('fts.')

    def merge_sc(self):
        self.fts = self.fts.merge(self.hist, left_on='fts.url', right_on='sc.fts_url')

    def extract_flows(self):
        """take all flows and add to df"""

        flow_df = pd.DataFrame()
        for v in self.fts.iterrows():
            norm = json_normalize(v[1]['fts.flows'])
            norm['fts.uid'] = v[1]['fts.iso3'] + v[1]['fts.year']
            norm['fts.sector'] = v[1]['fts.sector']

            if len(flow_df) == 0:
                flow_df = norm

            else:
                # need to make sure we're adding t3h same c0lz
                assert (norm.columns.all(flow_df.columns))
                flow_df = flow_df.append(norm, ignore_index=True)

        return flow_df

    def get_flow_bdown(self):
        """break down flows by needed columns"""

        def hp(v):
            if 'Plan' in json_normalize(v['destinationObjects']).values or 'Plan' in json_normalize(
                    v['sourceObjects']).values:
                return 'sc_has_plan'
            else:
                return 'sc_no_plan'

        #not graceful, should be better
        self.flows['plan'] = self.flows.apply(lambda x: hp(x), axis=1)

        # we are skipping boundary for now as it's <<<<
        d = self.flows[['fts.uid', 'fts.sector', 'amountUSD']].groupby(['fts.uid', 'fts.sector'], as_index=False).aggregate(
            {'amountUSD': 'sum'})
        piv = pd.pivot_table(d, index='fts.uid', values='amountUSD', aggfunc=np.sum, columns=[d['fts.sector']] \
                             , fill_value=0).reset_index()

        #do shelter plan/no plan
        s_d = self.flows[['fts.uid', 'amountUSD', 'plan']].loc[self.flows['fts.sector'] == 'SHL']\
                                .groupby(['fts.uid', 'plan'], as_index=False).aggregate(
                                {'amountUSD': 'sum'})
        s_piv = pd.pivot_table(s_d, index='fts.uid', values='amountUSD', aggfunc=np.sum, columns=[s_d['plan']] \
                             , fill_value=0).reset_index()

        mrgd = piv.merge(s_piv, on = 'fts.uid', how = 'outer')

        mrgd.columns = ['fts.' + v.lower() if 'fts.' not in v else v.lower() for v in mrgd.columns]

        return mrgd


import sc_pull
sc = sc_pull.pull()
# s_f = fts(test=2, sc=sc)
all_f = fts(sc=sc, all_sector=True)
all_f.get_flow_bdown().to_csv('../d0cz/fts_out.csv')

loaded 227 SC entries
pulling for : https://api.hpc.tools/v1/public/fts/flow?countryISO3=PAK&year=2005 &globalClusterCode=AGR
pulled. num bad resps: 0


/Users/ewanog/Documents/work/code/venv/p3/lib/python3.6/site-packages/pandas/core/frame.py:6211: FutureWarning: Sorting because non-concatenation axis is not aligned. A future version
of pandas will change to not sort by default.

To accept the future behavior, pass 'sort=False'.

To retain the current behavior and silence the warning, pass 'sort=True'.

  sort=sort)
/Users/ewanog/Documents/work/code/venv/p3/lib/python3.6/site-packages/ipykernel_launcher.py:123: FutureWarning: elementwise comparison failed; returning scalar instead, but in the future will perform elementwise comparison


In [28]:
master = pd.read_csv('../d0cz/master_manual_merge_rm_hno_dups.csv')
add = pd.read_csv('../d0cz/hno_process_out.csv')
# add = pd.read_csv('../d0cz/super_fancy_data_processed_extravaganza_raison_d_apostrophe_etre.csv')

mrg = master.merge(add, left_on='sc.uid', right_on='hno_uid', how='left')

mrg

,sc.region,sc.region_url,sc.country,sc.country_url,sc.year,sc.response,sc.lead_agency,sc.disaster_type,sc.website_url,sc.colead_agency,...,hno_count_national_ngo,hno_count_national_government,hno_count_total,hno_iso3,hno_disaster_type,hno_year,hno_presence_type,hno_source_type,hno_ass_type,hno_uid
0,Asia-Pacific,http://sheltercluster.org/asiapacific,Pakistan,http://sheltercluster.org/asiapacific/pakistan,2005,Pakistan - Kashmir Earthquake,IOM,Natural Disaster,NaN,NaN,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
1,Africa,http://sheltercluster.org/africa,Democratic Republic of Congo,http://sheltercluster.org/africa/democratic-re...,2006,Democratic Republic of Congo - Conflict,UNICEF,Conflict,http://sheltercluster.org/response/democratic-...,CRS,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
2,Asia-Pacific,http://sheltercluster.org/asiapacific,Indonesia,http://sheltercluster.org/asiapacific/indonesia,2006,Indonesia - Yogyakarta Earthquake,IFRC,Natural Disaster,http://sheltercluster.org/response/yogyakarta-...,NaN,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
3,MENA,http://sheltercluster.org/mena,Lebanon,NaN,2006,Lebanon - Conflict,UNHCR,Conflict,NaN,NaN,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
4,Africa,http://sheltercluster.org/africa,Liberia,NaN,2006,Liberia - Conflict,UNHCR,Conflict,NaN,NaN,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
5,Asia-Pacific,http://sheltercluster.org/asiapacific,Philippines,http://sheltercluster.org/asiapacific/philippines,2006,Philippines - Typhoon Durian,IFRC,Natural Disaster,http://sheltercluster.org/response/typhoon-dur...,NaN,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
6,Africa,http://sheltercluster.org/africa,Somalia,http://sheltercluster.org/africa/somalia,2006,Somalia - Conflict,UNHCR,Conflict,http://sheltercluster.org/response/somalia,NaN,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
7,Africa,http://sheltercluster.org/africa,Uganda,NaN,2006,Uganda - Conflict,UNICEF,Conflict,NaN,NaN,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
8,Asia-Pacific,http://sheltercluster.org/asiapacific,Bangladesh,http://sheltercluster.org/asiapacific/bangladesh,2007,Bangladesh - Cyclone Sidr,IFRC,Natural Disaster,http://sheltercluster.org/response/cyclone-sid...,UNDP,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
9,Africa,http://sheltercluster.org/africa,Central African Republic,http://sheltercluster.org/africa/central-afric...,2007,Central African Republic - Conflict,UNHCR,Conflict,http://sheltercluster.org/response/central-afr...,ACTED,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN


In [44]:
import urllib.request, json

import grequests
import pandas as pd

def api_pull(urls, test = False):
    """pull down API contents, and use local history if testing
        test: boolean, if testing
         urls: list of URLS to pull response from
    """
    def exception_handler(request, exception):
        print('Bad URL for ' + str(request))

    print('pulling for : ' + str(urls[0]))

    resps = []
    rs = (grequests.get(ref) for ref in urls)
    resps += grequests.map(rs, exception_handler=exception_handler, size=25)

    good_resps = []
    bad_resps = []
    for r in resps:
        load = json.loads(r.content)
        load['url'] = r.url
        if r.status_code == 200:
            good_resps.append(load)
        else:
            bad_resps.append(load)

    print('pulled. num bad resps: ' + str(len(bad_resps)))

    return good_resps

In [47]:
r = api_pull(['https://api.hpc.tools/v1/public/global-cluster']) 

pulling for : https://api.hpc.tools/v1/public/global-cluster
pulled. num bad resps: 0


In [54]:
for v in r[0]['data']:
    print(v['code'] + ', ' +  v['name']) 

AGR, Agriculture
CCM, Camp Coordination / Management
PRO-CPN, Child Protection
CSS, Coordination and support services
ERY, Early Recovery
EDU, Education
SHL, Emergency Shelter and NFI
TEL, Emergency Telecommunications
FSC, Food Security
PRO-GBV, Gender Based Violence
HEA, Health
PRO-HLP, Housing, Land and Property
LOG, Logistics
PRO-MIN, Mine Action
MS, Multi-sector
NUT, Nutrition
OTH, Other
PRO, Protection
WSH, Water Sanitation Hygiene


In [29]:
#pull down RW types

r = api_pull(['https://api.reliefweb.int/v1/disasters/42784'])

pulling for : https://api.reliefweb.int/v1/disasters/42784
pulled. num bad resps: 0


In [42]:
r[0]['data'][0]['fields']

{'country': [{'href': 'https://api.reliefweb.int/v1/countries/31',
   'id': 31,
   'iso3': 'bgd',
   'location': {'lat': 23.84, 'lon': 90.27},
   'name': 'Bangladesh',
   'primary': True},
  {'href': 'https://api.reliefweb.int/v1/countries/119',
   'id': 119,
   'iso3': 'ind',
   'location': {'lat': 22.76, 'lon': 79.28},
   'name': 'India'},
  {'href': 'https://api.reliefweb.int/v1/countries/165',
   'id': 165,
   'iso3': 'mmr',
   'location': {'lat': 21.15, 'lon': 96.51},
   'name': 'Myanmar'}],
 'current': False,
 'date': {'created': '2017-05-27T00:00:00+00:00'},
 'description': "Severe Cyclonic Storm Mora made landfall in **Bangladesh** on 30 May 2017. The United Nations Institute for Training and Research (UNITAR)-Operational Satellite Applications Programme (UNOSAT) estimates that 10,074,699 people were living in regions that were exposed to wind speeds of 120 km/h or more, and that some districts near Cox’s Bazaar and Chittagong were hit with gusts that topped 130 km/h. \r\n\r\nT

In [27]:
for v in r[0]['data']:
    print(v['fields']['code'] + ',' + v['fields']['name'])

CE,Complex Emergency
FR,Fire
SS,Storm Surge
IN,Insect Infestation
OT,Other
HT,Heat Wave
MS,Mud Slide
AV,Snow Avalanche
LS,Land Slide
EC,Extratropical Cyclone
TS,Tsunami
DR,Drought
ST,Severe Local Storm
CW,Cold Wave
WF,Wild Fire
EP,Epidemic
EQ,Earthquake
FF,Flash Flood
AC,Technological Disaster
TC,Tropical Cyclone
VO,Volcano
FL,Flood


In [12]:
mrg['sc.uid']

0      PAK2005
1      COD2006
2      IDN2006
3      LBN2006
4      LBR2006
5      PHL2006
6      SOM2006
7      UGA2006
8      BGD2007
9      CAF2007
10     TCD2007
11     COD2007
12     ETH2007
13     MOZ2007
14     PAK2007
15     SOM2007
16     AFG2008
17     CAF2008
18     TCD2008
19     COD2008
20     GEO2008
21     HTI2008
22     IRQ2008
23     KEN2008
24     MOZ2008
25     MMR2008
26     NPL2008
27     PAK2008
28     SOM2008
29     SDN2008
        ...   
449    SDN2018
450    SDN2018
451    SDN2018
452    SDN2018
453    SYR2018
454    SYR2018
455    SYR2018
456    SYR2018
457    SYR2018
458    SYR2018
459    UKR2018
460    UKR2018
461    UKR2018
462    UKR2018
463    YEM2018
464    YEM2018
465    YEM2018
466    YEM2018
467    VUT2018
468    PHL2018
469    TON2018
470    SLV2018
471    NGA2018
472    NGA2018
473    NGA2018
474    NGA2018
475    NGA2018
476    NGA2018
477    MDG2018
478    MOZ2018
Name: sc.uid, Length: 479, dtype: object

In [2]:
df.to_csv('df.csv')

NameError: name 'df' is not defined

In [8]:
df.apply(lambda x : x)

self.data[['rw_gen.month', 'rw_gen.year']] = self.data.apply(lambda x:
                                                             pd.Series(
                                                                 self.extract_date(x['rw_gen.fields.name'])),
                                                             axis=1)


NameError: name 'df' is not defined

In [71]:
l = []

df.apply(lambda x : l.append(x))

l

[0    None
 1    None
 2    None
 Name: col2, dtype: object, 0    None
 1    None
 2    None
 Name: col2, dtype: object]

In [77]:
pd.concat([df,df])

df.con

AttributeError: 'DataFrame' object has no attribute 'concat'

In [42]:
df.apply(lambda x : x)


urls = []
for v in df.iterrows():
    print(v[1]['col1'])

1
3
None


In [29]:
set(['a', 'b']).issubset(['a', 'b', 'c'])

True

In [35]:
df

,col1,col2
0,1,2
1,3,None
2,None,3


In [36]:
m

,col1,new
0,1,


In [54]:
m

,col1,new
0,1,42
1,x,None
2,x,None
3,3,NaN


In [61]:
def s(x):
    return '12'

df[df.col1.apply(lambda x : s(x)) == '1']

,col1,col2


In [4]:
df[df.col1 == '1'].col2 = 100

df.loc[df.col1 == '1', 'col1'] = "Matt"

df

/Users/ewanog/Documents/work/code/venv/p3/lib/python3.6/site-packages/pandas/core/generic.py:4405: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: http://pandas.pydata.org/pandas-docs/stable/indexing.html#indexing-view-versus-copy
  self[name] = value


,col1,col2
0,Matt,42
1,3,44
2,None,3


In [19]:
m

,col1,new
0,1,123
1,x,
2,x,
3,3,


In [20]:
df

,col1,col2
0,1,42
1,3,44
2,None,3


In [29]:
df.update(m)

df

,col1,col2
0,1,123
1,x,
2,x,


In [28]:
d = {'col1': ['1','3', None], 'col2': ['42','44', '3']}
m = {'col1': ['1','x','x','3'], 'col2' : ['123','','','']}
df = pd.DataFrame(data=d)
m = pd.DataFrame(data=m)

def find(v):
    if v in df.col1.values:
        return df[df.col1 == v].col2

# m.new = m.apply(lambda x : find(x.col1), axis = 1)

In [81]:
d = {'col1': [np.nan,'3', 4, '3'], 'col2': [np.nan, np.nan, '3', np.nan]}
df = pd.DataFrame(data=d)

df.col1[df.col1 == 19] = 19000
df

,col1,col2
0,NaN,NaN
1,3,NaN
2,4,3
3,3,NaN


In [52]:
s.index

Index([], dtype='object')

In [73]:
df

,col1,col2
0,NaN,NaN
1,3,NaN
2,4,3
3,3,1


In [82]:
null_cnt = {}

UID_COL = 'col1'
def find_empt(r):
    num_empt = sum(r.isnull())
    if r[UID_COL] in null_cnt:
        null_cnt[r[UID_COL]] += num_empt
        
    else:
        null_cnt[r[UID_COL]] = num_empt
    
df.apply(lambda x : find_empt(x), axis = 1)

0    None
1    None
2    None
3    None
dtype: object

In [83]:
null_cnt

{nan: 2, '3': 2, 4: 0}

In [9]:
globals()['In']

['',
 "d = {'col1': ['1','3', None], 'col2': ['2',None, '3']}\ndf = pd.DataFrame(data=d)\n\ndf\n\ndef a(r):\n    print(r)\n    print('**')\n    return -99\n    \ndf.col1 = df.col1.apply(lambda x : a(x), axis = 1)\n\nd['new']",
 "d = {'col1': ['1','3', None], 'col2': ['2',None, '3']}\ndf = pd.DataFrame(data=d)\n\ndf",
 'import pandas as pd',
 "d = {'col1': ['1','3', None], 'col2': ['2',None, '3']}\ndf = pd.DataFrame(data=d)\n\ndf",
 "d = {'col1': ['1','3', None], 'col2': ['2',None, '3']}\ndf = pd.DataFrame(data=d)\n\ndf",
 "d = {'col1': ['1','3', None], 'col2': ['2',None, '3']}\ndf = pd.DataFrame(data=d)\n\ndf = pd.DataFrame(df['col1'])",
 "d = {'col1': ['1','3', None], 'col2': ['2',None, '3']}\ndf = pd.DataFrame(data=d)\n\ndf = pd.DataFrame(df['col1'])\n\ndf",
 'globals()',
 "globals()['In']"]

In [25]:
df

,col1,col2
0,-99,2
1,-99,None
2,-99,3


In [50]:
class c(object):
    def __init__(self):
        self.a = 1
        
    def r(self):
        print(1)
        
    def main(self):
        self.r()

s = c()
s.main()

1


In [62]:
df = pd.DataFrame(data = {'a': [1, 2, 3], 'b': [4, 5, 6]})
ot = pd.DataFrame(data = {'a': [6, 3, 1], 'c': [14, 15, 16]})

df.merge(ot, how = 'left', left_on='a', right_on='a')

,a,b,c
0,1,4,16.0
1,2,5,NaN
2,3,6,15.0


In [121]:
'a'.

v = 'text'

v.

In [120]:
{1 : 2 if True 1 if False}

SyntaxError: invalid syntax (<ipython-input-120-2b5f64ea7908>, line 1)

In [111]:
d

['Since its emergence in April 2009, pandemic influenza A(H1N1) 2009 virus affected all parts of the world, causing the first influenza pandemic since 1968. Following the observation that 1 or 2 epidemic periods had occurred in most countries and in most WHO Regions, WHO declared on 10 Aug 2010 that the world was moving into the post-pandemic period. ([WHO, 18 Feb 2011](https://reliefweb.int/node/389128))',
 'https://reliefweb.int/disaster/ep-2009-000084-mex',
 'World',
 'wld',
 'https://api.reliefweb.int/v1/countries/254']

In [108]:
d = ['Since its emergence in April 2009, pandemic influenza A(H1N1) 2009 virus affected all parts of the world, causing the first influenza pandemic since 1968. Following the observation that 1 or 2 epidemic periods had occurred in most countries and in most WHO Regions, WHO declared on 10 Aug 2010 that the world was moving into the post-pandemic period. ([WHO, 18 Feb 2011](https://reliefweb.int/node/389128))', 'https://reliefweb.int/disaster/ep-2009-000084-mex', 'World', 'wld', 'https://api.reliefweb.int/v1/countries/254']

In [104]:
fin = json.loads(urllib.request.urlopen("https://api.reliefweb.int/v1/" \
                           "disasters?appname=vocabulary&preset=external").read().decode())['count']

fin

10

In [91]:
import urllib.request, json

with urllib.request.urlopen("https://api.reliefweb.int/v1/disasters?appname=vocabulary&preset=external&limit=10&offset=1000".format(i*1000)) as url:
    j = json.loads(url.read().decode())
    
j.keys()

dict_keys(['href', 'time', 'links', 'totalCount', 'count', 'data'])

In [56]:
import urllib.request, json

from pandas.io.json import json_normalize

r = json_normalize(json.loads(urllib.request.urlopen("https://api.hpc.tools/v1/public/global-cluster")
                                              .read().decode()))['data']

r = [v['code'] for v in r[0]]
r.remove('SHL')
s = ','.join(r)

s
# https://api.hpc.tools/v1/public/fts/flow?countryISO3=SOM&year=2015&globalClusterCode=shl,tel

'AGR,CCM,PRO-CPN,CSS,ERY,EDU,TEL,FSC,PRO-GBV,HEA,PRO-HLP,LOG,PRO-MIN,MS,NUT,OTH,PRO,WSH'

In [54]:
json_normalize(json.loads(urllib.request.urlopen("https://api.hpc.tools/v1/public/fts/flow?countryISO3=SOM&year=2015&globalClusterCode=shl,wsh")
                                              .read().decode()))

,data.flows,data.incoming.flowCount,data.incoming.fundingTotal,data.incoming.pledgeTotal,data.internal.flowCount,data.outgoing.flowCount,data.outgoing.fundingTotal,data.outgoing.pledgeTotal,meta.count,meta.language,status
0,"[{'id': '126486', 'versionId': 1, 'description...",52,38478845,0,0,6,2400471,0,58,en,ok


In [57]:
from fts_pull import fts




loaded 227 SC entries
pulled countries
num bad resps: 0


/Users/ewanog/Documents/work/code/venv/p3/lib/python3.6/site-packages/pandas/core/frame.py:6201: FutureWarning: Sorting because non-concatenation axis is not aligned. A future version
of pandas will change to not sort by default.

To accept the future behavior, pass 'sort=True'.

To retain the current behavior and silence the warning, pass sort=False

  sort=sort)


In [59]:
from sc_pull import pull

fts(sc = pull(), test = 10, all_sector=True)

loaded 227 SC entries


TypeError: 'fts' object is not callable

In [22]:
s = pd.Series([1,2,312312414312312312312414312312414312312414312312414414312312414312312414312312414,23])

print(s.values)

[1 2
 312312414312312312312414312312414312312414312312414414312312414312312414312312414
 23]


In [4]:
json.loads(urllib.request.urlopen("https://api.hpc.tools/v1/public/location")
                                              .read().decode())

{'data': [{'adminLevel': 0,
   'id': 1,
   'iso3': 'AFG',
   'name': 'Afghanistan',
   'pcode': None},
  {'adminLevel': 0, 'id': 3, 'iso3': 'ALB', 'name': 'Albania', 'pcode': None},
  {'adminLevel': 0, 'id': 4, 'iso3': 'DZA', 'name': 'Algeria', 'pcode': None},
  {'adminLevel': 0,
   'id': 5,
   'iso3': 'ASM',
   'name': 'American Samoa',
   'pcode': None},
  {'adminLevel': 0, 'id': 6, 'iso3': 'AND', 'name': 'Andorra', 'pcode': None},
  {'adminLevel': 0, 'id': 7, 'iso3': 'AGO', 'name': 'Angola', 'pcode': None},
  {'adminLevel': 0,
   'id': 8,
   'iso3': 'AIA',
   'name': 'Anguilla (United Kingdom)',
   'pcode': None},
  {'adminLevel': 0,
   'id': 9,
   'iso3': 'ATA',
   'name': 'Antarctica',
   'pcode': None},
  {'adminLevel': 0,
   'id': 10,
   'iso3': 'ATG',
   'name': 'Antigua and Barbuda',
   'pcode': None},
  {'adminLevel': 0,
   'id': 11,
   'iso3': 'ARG',
   'name': 'Argentina',
   'pcode': None},
  {'adminLevel': 0, 'id': 12, 'iso3': 'ARM', 'name': 'Armenia', 'pcode': None},
  {

In [3]:
import urllib.request, json
import csv
import itertools
import math
import collections
import sys

import grequests
import pandas as pd
import numpy as np
from pandas.io.json import json_normalize

In [68]:
from itertools import groupby

v = sorted([('b',1), ('b', 2), ('a', 5)], key = lambda x: x[0])

c_v = {}
for k, group in groupby(v, lambda x : x[0]):
    c_v[k] = [v[1] for v in group]

c_v
    
#sort aggd
# aggd = sorted(aggd, key=lambda k: k[:4])


{'a': [5], 'b': [1, 2]}

In [ ]:
"""for pulling down data from hr.info API on response level and sending to CSV"""
#locations: https://api.hpc.tools/v1/public/location
#clusters: https://api.hpc.tools/v1/public/global-cluster ["code":"SHL"]
#https://api.hpc.tools/docs/v1/
#boundary cat by looping through locations

#TODO: subtract outgoing

import urllib.request, json
import re
from itertools import groupby

import pandas as pd
import numpy as np
from pandas.io.json import json_normalize

import utils


class fts(object):
    def __init__(self, sc, test=None, all_sector=False):
        self.all_sector = all_sector
        self.test = test
        self.sc = sc

        self.fts = self.master_pull()
        self.merge_sc()
        self.flows = self.extract_flows()

        # remove flows col as it's quite large
        self.fts = self.fts.drop('fts.data.flows', axis=1)

    def master_pull(self):
        """group by year, country and pull down total values with optional filter by cluster.
            returns: dict of {sector 1 : info df}
        """
        countries = json_normalize(json.loads(urllib.request.urlopen("https://api.hpc.tools/v1/public/location")
                                              .read().decode()))['data']

        print('pulled countries')

        if self.test:
            self.hist = self.sc[:self.test]
        else:
            self.hist = self.sc

        #create cluster_vals which will be a dict of {cluster : df}
        URL_STR = 'https://api.hpc.tools/v1/public/fts/flow?countryISO3={0}&year={1} &globalClusterCode={2}'
        cluster_vals = {'SHL': URL_STR}
        if not self.all_sector:
            #get extract all sector keywords from API
            cluster_vals = {k: URL_STR for k in [v['code'] for v in json_normalize(json.loads \
                                        (urllib.request.urlopen("https://api.hpc.tools/v1/public/global-cluster")
                                        .read().decode()))['data'][0]]}

        print(cluster_vals)

        #for each URL, make call for each cluster
        hrefz = []
        for k in cluster_vals.keys():
            self.hist.apply(lambda v: hrefz.append(cluster_vals[k].format(v['sc.iso3'], v['sc.year'], k)), axis = 1)

        #now assign the df'd vals to the appropriate cluster responses
        for k, group in groupby([(,
                                  json_normalize(v['data'])) for v in utils.api_pull(hrefz, False)], lambda x: x[0]):
            cluster_vals[k] = [v[1] for v in group]

        #do one big pull with all of the sector specific URLs, then assign to cluster_values using cluser code in URL
        return json_normalize(resps).add_prefix('fts.')

    def merge_sc(self):
        self.fts = self.fts.merge(self.hist, left_on='fts.url', right_on='sc.fts_url')

    def extract_flows(self):
        """take all flows and add to df"""

        flow_df = pd.DataFrame()
        for v in self.fts.iterrows():
            norm = json_normalize(v[1]['fts.data.flows'])
            norm['fts.uid'] = v[1]['sc.uid']

            if len(flow_df) == 0:
                flow_df = norm
            else:
                # need to make sure we're adding t3h same c0lz
                assert (norm.columns.all(flow_df.columns))
                flow_df = flow_df.append(norm, ignore_index=True)

        return flow_df

    def get_flow_bdown(self):
        """break down flows by needed columns"""

        def hp(v):
            if 'Plan' in json_normalize(v['destinationObjects']).values or 'Plan' in json_normalize(
                    v['sourceObjects']).values:
                return 'has_plan'
            else:
                return 'no_plan'

        self.flows['plan'] = self.flows.apply(lambda x: hp(x), axis=1)

        # we are skipping boundary for now as it's <<<<
        d = self.flows[['fts.uid', 'plan', 'amountUSD']].groupby(['fts.uid', 'plan'], as_index=False).aggregate(
            {'amountUSD': 'sum'})
        piv = pd.pivot_table(d, index='fts.uid', values='amountUSD', aggfunc=np.sum, columns=[d['plan']] \
                             , fill_value=0).reset_index()
        piv['total'] = piv['has_plan'] + piv['no_plan']
        return piv

In [227]:

import sc_pull
sc = sc_pull.pull()
s_f = fts(test=2, sc=sc, all_sector=True)
# all_f = fts(test=5, sc=sc, all_sector=True)

print(s_f.get_flow_bdown().add_prefix('sc_')) #351513201 PAK2005 sc
# print(all_f.get_flow_bdown().add_prefix('sc_')) #351513201 PAK2005 all

# AGR,CCM,PRO-CPN,CSS,ERY,EDU,TEL,FSC,PRO-GBV,HEA,PRO-HLP,LOG,PRO-MIN,MS,NUT,OTH,PRO,WSH


loaded 227 SC entries
['AGR', 'CCM', 'PRO-CPN', 'CSS', 'ERY', 'EDU', 'SHL', 'TEL', 'FSC', 'PRO-GBV', 'HEA', 'PRO-HLP', 'LOG', 'PRO-MIN', 'MS', 'NUT', 'OTH', 'PRO', 'WSH']
pulling for : https://api.hpc.tools/v1/public/fts/flow?countryISO3=PAK&year=2005 &globalClusterCode=AGR
pulled. num bad resps: 0


/Users/ewanog/Documents/work/code/venv/p3/lib/python3.6/site-packages/pandas/core/frame.py:6201: FutureWarning: Sorting because non-concatenation axis is not aligned. A future version
of pandas will change to not sort by default.

To accept the future behavior, pass 'sort=True'.

To retain the current behavior and silence the warning, pass sort=False

  sort=sort)


fts.sector sc_fts.uid   sc_AGR   sc_CCM     sc_CSS    sc_EDU    sc_ERY  \
0             COD2006  5236489        0   45656350  11563291   5778196   
1             PAK2005  7243295  5190601  151931349  33029232  30881131   

fts.sector     sc_FSC     sc_HEA    sc_LOG      sc_MS   sc_NUT    sc_PRO  \
0           140051048   86679863  21634059   80137878  5586465  22822419   
1            90471299  145035185   9829858  323416603        0   7195087   

fts.sector  sc_PRO-MIN     sc_SHL  sc_TEL    sc_WSH  
0              5727948    8475684   68020  17236222  
1               423729  115218527  597721  58147247  


In [90]:
l = []

l.append(1,2)

TypeError: append() takes exactly one argument (2 given)

In [312]:
clusters = [(v['code'], v['name']) for v in json_normalize(json.loads \
                                                  (urllib.request.urlopen(
                                                    "https://api.hpc.tools/v1/public/global-cluster")
                                                    .read().decode()))['data'][0]]

for v in clusters:
    print(v[0] + ' : ' + v[1])

AGR : Agriculture
CCM : Camp Coordination / Management
PRO-CPN : Child Protection
CSS : Coordination and support services
ERY : Early Recovery
EDU : Education
SHL : Emergency Shelter and NFI
TEL : Emergency Telecommunications
FSC : Food Security
PRO-GBV : Gender Based Violence
HEA : Health
PRO-HLP : Housing, Land and Property
LOG : Logistics
PRO-MIN : Mine Action
MS : Multi-sector
NUT : Nutrition
OTH : Other
PRO : Protection
WSH : Water Sanitation Hygiene


In [302]:
"""for pulling down data from hr.info API on response level and sending to CSV"""
#locations: https://api.hpc.tools/v1/public/location
#clusters: https://api.hpc.tools/v1/public/global-cluster ["code":"SHL"]
#https://api.hpc.tools/docs/v1/
#boundary cat by looping through locations

#TODO: subtract outgoing

import urllib.request, json
import re
from itertools import groupby

import pandas as pd
import numpy as np
from pandas.io.json import json_normalize

import utils


class fts(object):
    def __init__(self, sc, test=None, all_sector=False):
        self.all_sector = all_sector
        self.test = test
        self.sc = sc

        self.fts = self.master_pull()
        # self.merge_sc()
        self.flows = self.extract_flows()

        # remove flows col as it's quite large
#         self.fts = self.fts.drop('fts.data.flows', axis=1)

    def master_pull(self):
        """group by year, country and pull down total values with optional filter by cluster.
            returns: dict of {sector 1 : info df}
        """
        SHELTER_ABBRV = 'SHL'

        if self.test:
            self.hist = self.sc[:self.test]
        else:
            self.hist = self.sc

        #create cluster_vals which will be a dict of {cluster : df}
        """
            AGR : Agriculture
            CCM : Camp Coordination / Management
            PRO-CPN : Child Protection
            CSS : Coordination and support services
            ERY : Early Recovery
            EDU : Education
            SHL : Emergency Shelter and NFI
            TEL : Emergency Telecommunications
            FSC : Food Security
            PRO-GBV : Gender Based Violence
            HEA : Health
            PRO-HLP : Housing, Land and Property
            LOG : Logistics
            PRO-MIN : Mine Action
            MS : Multi-sector
            NUT : Nutrition
            OTH : Other
            PRO : Protection
            WSH : Water Sanitation Hygiene
        """
        clusters = [SHELTER_ABBRV]
        if self.all_sector:
            clusters = [v['code'] for v in json_normalize(json.loads \
                                                              (urllib.request.urlopen(
                                                                "https://api.hpc.tools/v1/public/global-cluster")
                                                                .read().decode()))['data'][0]]

        print('**' + str(clusters))

        #for each cluster, prep string for each URL and get year
        URL_STR = 'https://api.hpc.tools/v1/public/fts/flow?countryISO3={0}&year={1} &globalClusterCode={2}'
        hrefz = []
        for clust in clusters:
            self.hist.apply(lambda v: hrefz.append(URL_STR.format(v['sc.iso3'], v['sc.year'], clust)) , axis = 1)

        # now submit the URLs to the API service and concat together and
        # then assign to cluster_values using cluser code in URL
        pulled = None
        for v in utils.api_pull(hrefz, False):
            ins_df = json_normalize(v['data'])

            ins_df['sector'] = re.match('.*globalClusterCode=(.*)', v['url']).groups()[0]
            ins_df['year'] = re.match('.*year=([0-9]{4})', v['url']).groups()[0]
            ins_df['iso3'] = re.match('.*countryISO3=(.{3})', v['url']).groups()[0]

            if pulled is None:
                pulled = ins_df

            else:
                pulled = pd.concat([pulled, ins_df])

        return pulled.add_prefix('fts.')

    def merge_sc(self):
        self.fts = self.fts.merge(self.hist, left_on='fts.url', right_on='sc.fts_url')

    def extract_flows(self):
        """take all flows and add to df"""

        flow_df = pd.DataFrame()
        for v in self.fts.iterrows():
            norm = json_normalize(v[1]['fts.flows'])
            norm['fts.uid'] = v[1]['fts.iso3'] + v[1]['fts.year']
            norm['fts.sector'] = v[1]['fts.sector']

            if len(flow_df) == 0:
                flow_df = norm

            else:
                # need to make sure we're adding t3h same c0lz
                assert (norm.columns.all(flow_df.columns))
                flow_df = flow_df.append(norm, ignore_index=True)
        
        print('**' + str(flow_df.columns))
        return flow_df

    def get_flow_bdown(self):
        """break down flows by needed columns"""

        def hp(v):
            if 'Plan' in json_normalize(v['destinationObjects']).values or 'Plan' in json_normalize(
                    v['sourceObjects']).values:
                return 'sc_has_plan'
            else:
                return 'sc_no_plan'
        
        #not graceful, should be better
        self.flows['plan'] = self.flows.apply(lambda x: hp(x), axis=1)

        # we are skipping boundary for now as it's <<<<
        d = self.flows[['fts.uid', 'fts.sector', 'amountUSD']].groupby(['fts.uid', 'fts.sector'], as_index=False).aggregate(
            {'amountUSD': 'sum'})
        piv = pd.pivot_table(d, index='fts.uid', values='amountUSD', aggfunc=np.sum, columns=[d['fts.sector']] \
                             , fill_value=0).reset_index()
        
        #do shelter plan/no plan
        s_d = self.flows[['fts.uid', 'amountUSD', 'plan']].loc[self.flows['fts.sector'] == 'SHL']\
                                .groupby(['fts.uid', 'plan'], as_index=False).aggregate(
                                {'amountUSD': 'sum'})
        s_piv = pd.pivot_table(s_d, index='fts.uid', values='amountUSD', aggfunc=np.sum, columns=[s_d['plan']] \
                             , fill_value=0).reset_index()
        
        mrgd = piv.merge(s_piv, on = 'fts.uid', how = 'outer')
        
        mrgd.columns = ['fts.' + v.lower() if 'fts.' not in v else v.lower() for v in mrgd.columns]
        
        return mrgd

In [303]:
import utils
import re

import sc_pull
sc = sc_pull.pull()
s_f = fts(test=2, sc=sc, all_sector=True)
r = s_f.get_flow_bdown()

loaded 227 SC entries
**['AGR', 'CCM', 'PRO-CPN', 'CSS', 'ERY', 'EDU', 'SHL', 'TEL', 'FSC', 'PRO-GBV', 'HEA', 'PRO-HLP', 'LOG', 'PRO-MIN', 'MS', 'NUT', 'OTH', 'PRO', 'WSH']
pulling for : https://api.hpc.tools/v1/public/fts/flow?countryISO3=PAK&year=2005 &globalClusterCode=AGR
pulled. num bad resps: 0


/Users/ewanog/Documents/work/code/venv/p3/lib/python3.6/site-packages/pandas/core/frame.py:6201: FutureWarning: Sorting because non-concatenation axis is not aligned. A future version
of pandas will change to not sort by default.

To accept the future behavior, pass 'sort=True'.

To retain the current behavior and silence the warning, pass sort=False

  sort=sort)


**Index(['amountUSD', 'boundary', 'budgetYear', 'contributionType', 'createdAt',
       'date', 'decisionDate', 'description', 'destinationObjects',
       'exchangeRate', 'firstReportedDate', 'flowType', 'fts.sector',
       'fts.uid', 'id', 'keywords', 'method', 'newMoney', 'onBoundary',
       'originalAmount', 'originalCurrency', 'parentFlowId', 'refCode',
       'reportDetails', 'sourceObjects', 'status', 'updatedAt', 'versionId'],
      dtype='object')


In [304]:
r

,fts.uid,fts.agr,fts.ccm,fts.css,fts.edu,fts.ery,fts.fsc,fts.hea,fts.log,fts.ms,fts.nut,fts.pro,fts.pro-min,fts.shl,fts.tel,fts.wsh,fts.sc_has_plan,fts.sc_no_plan
0,COD2006,5236489,0,45656350,11563291,5778196,140051048,86679863,21634059,80137878,5586465,22822419,5727948,8475684,68020,17236222,6172648,2303036
1,PAK2005,7243295,5190601,151931349,33029232,30881131,90471299,145035185,9829858,323416603,0,7195087,423729,115218527,597721,58147247,21137104,94081423


In [246]:
s_f.get_flow_bdown().head(1)[['AGR', 'CCM', 'CSS', 'EDU', 'ERY', 'FSC', 'HEA', 'LOG', 'MS',
       'NUT', 'PRO', 'PRO-MIN', 'SHL', 'TEL', 'WSH']].values.sum()

# s_f.get_flow_bdown().head(1)

# COD06: 456653932
        448,589,418
# PAK05: 1435264796

1,255,216,413
1,435,264,796

456653932